# Análisis de los incritos en el Cabildo de Científicxs

### FCFM, 4 de noviembre de 2019

In [1]:
import pandas as pd
import numpy as np
import re
import nltk

import warnings
warnings.filterwarnings("ignore")

In [2]:
datos = pd.read_excel("inscritosCabildo_sinRUT.xlsx")

In [3]:
datos.shape

(186, 6)

In [4]:
datos.sample(3)

,Timestamp,Edad,Sexo,Etapa en la que estás en tu carrera,Facultad en la que trabajas (e.g. FCFM),Universidad a la que pertences (e.g U.Chile)
167,2019-11-04 15:33:31.022,23,Femenino,Estudiante,Fcfm,Universidad de chile
130,2019-11-04 14:25:15.307,40,Masculino,Investigador,Fundación Juan Enrique Lagarrigue,Egresado Universidad Arcis
52,2019-11-04 14:13:41.272,26,Femenino,Estudiante,Faciqyf,U. Chile


In [5]:
datos.columns = ['timestamp', 'edad', 'sexo', 'posicion', 'lugar', 'institucion']

In [6]:
list(datos.columns) 

['timestamp', 'edad', 'sexo', 'posicion', 'lugar', 'institucion']

In [8]:
pd.DataFrame(datos.isnull().sum() * 100 / len(datos))

,0
timestamp,0.000000
edad,0.000000
sexo,0.000000
posicion,0.000000
lugar,6.451613
institucion,2.688172


In [10]:
datos['lugar'] = datos['lugar'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
datos['lugar'] = datos['lugar'].str.lower().str.strip() # todo en caps off y fuera espacios

datos['lugarNormalizado'] = datos['lugar'].replace(["ingenieria", "fcfm", "beauchef", "fcfm-dfi", "cmm"] , "Ingenieria")

datos['lugarNormalizado'] = datos['lugarNormalizado'].replace(["facultad de ciencias quimicas y farmaceuticas",
     "fcqyf", "faciqyf", "fciqyf", "fcqyf", "ciencias quimicas y farmaceuticas", "fac de ciencias quimicas y farmaceuticas"] , "FCQYF")

datos['lugarNormalizado'] = datos['lugarNormalizado'].replace(["medicina",
     "icbm - facultad de medicina", "medicina esp", "facultad de medicina"] , "FMED")


datos['institucion'] = datos['institucion'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
datos['institucion'] = datos['institucion'].str.lower().str.strip() 

datos['institucionNormalizada'] = datos['institucion'].replace(["uchile", "u de chile", "u. chile", 
     "u chile", "u. de chile", "u.chile", "fcfm", "universidad de chile", "uch"] , "Uchile")

datos['institucionNormalizada'] = datos['institucionNormalizada'].replace(["puc", "universidad catolica de chile", "uc"] , "PUC")
datos['institucionNormalizada'] = datos['institucionNormalizada'].replace(["universidad andres bello", 
     "universidad nacional andres bello", "andres bello", "unab"] , "UNAB")

datos['institucionNormalizada'] = datos['institucionNormalizada'].replace(["u mayor", 
     "universidad mayor"] , "UMayor")

In [12]:
datos.sample(10)

,timestamp,edad,sexo,posicion,lugar,institucion,lugarNormalizado,institucionNormalizada
33,2019-11-04 14:12:45.355,23,Femenino,Estudiante,fcfm,u.chile,Ingenieria,Uchile
38,2019-11-04 14:13:06.835,25,Femenino,Estudiante,fcfm,uchile,Ingenieria,Uchile
97,2019-11-04 14:16:54.774,32,Masculino,Otro / No Aplica,fcfm,u.chile,Ingenieria,Uchile
5,2019-11-04 14:10:00.121,18,Masculino,Estudiante,NaN,fcfm,NaN,Uchile
32,2019-11-04 14:12:42.560,22,Femenino,Estudiante,fcfm,u.chile,Ingenieria,Uchile
56,2019-11-04 14:13:44.431,23,Masculino,Estudiante,fcfm-dfi,u chile,Ingenieria,Uchile
35,2019-11-04 14:12:50.839,23,Femenino,Estudiante,fcfm,uchile,Ingenieria,Uchile
18,2019-11-04 14:11:34.584,23,Masculino,Estudiante,fcqyf,u.chile,FCQYF,Uchile
150,2019-11-04 14:39:45.733,19,Masculino,Estudiante,fcfm,u.chile,Ingenieria,Uchile
4,2019-11-04 14:09:24.549,38,Masculino,Académico,fcfm,u. chile,Ingenieria,Uchile


## Estadísticas sumarias

In [32]:
datos['edad'].describe()

count    186.000000
mean      27.010753
std        8.997290
min       18.000000
25%       21.250000
50%       24.000000
75%       29.750000
max       72.000000
Name: edad, dtype: float64

In [36]:
datos['sexo'].value_counts(normalize=False)

Masculino    117
Femenino      68
Otro           1
Name: sexo, dtype: int64

In [37]:
datos['sexo'].value_counts(normalize=True)

Masculino    0.629032
Femenino     0.365591
Otro         0.005376
Name: sexo, dtype: float64

In [38]:
datos['posicion'].value_counts(normalize=False)

Estudiante          132
Investigador         21
Académico            19
Otro / No Aplica     14
Name: posicion, dtype: int64

In [39]:
datos['posicion'].value_counts(normalize=True)

Estudiante          0.709677
Investigador        0.112903
Académico           0.102151
Otro / No Aplica    0.075269
Name: posicion, dtype: float64

In [40]:
datos['lugarNormalizado'].value_counts(normalize=False)

Ingenieria                               128
FCQYF                                     17
FMED                                       6
ciencias de la vida                        1
ninguna                                    1
fcfm geologia                              1
facultad de ciencias biologicas uc         1
facultad de ingenieria                     1
geologia unab                              1
farmacia                                   1
artes                                      1
ffhh                                       1
facultad de ciencias de la vida            1
geologia                                   1
ciencias biologicas                        1
facultad de arquitectura                   1
fcfm - fmed                                1
na                                         1
ciencias veterinarias y pecuarias          1
escuela de medicina uc                     1
gema                                       1
fundacion juan enrique lagarrigue          1
salud     

In [41]:
datos['lugarNormalizado'].value_counts(normalize=True)

Ingenieria                               0.735632
FCQYF                                    0.097701
FMED                                     0.034483
ciencias de la vida                      0.005747
ninguna                                  0.005747
fcfm geologia                            0.005747
facultad de ciencias biologicas uc       0.005747
facultad de ingenieria                   0.005747
geologia unab                            0.005747
farmacia                                 0.005747
artes                                    0.005747
ffhh                                     0.005747
facultad de ciencias de la vida          0.005747
geologia                                 0.005747
ciencias biologicas                      0.005747
facultad de arquitectura                 0.005747
fcfm - fmed                              0.005747
na                                       0.005747
ciencias veterinarias y pecuarias        0.005747
escuela de medicina uc                   0.005747


In [42]:
datos['institucionNormalizada'].value_counts(normalize=False)

Uchile                        157
UNAB                            8
PUC                             6
UMayor                          2
universidad de valparaiso       1
universidad de la serena        1
universidad de concepcion       1
titulada uandes                 1
me pertenezco a mi              1
egresado universidad arcis      1
duoc uc                         1
chile                           1
Name: institucionNormalizada, dtype: int64

In [43]:
datos['institucionNormalizada'].value_counts(normalize=True)

Uchile                        0.867403
UNAB                          0.044199
PUC                           0.033149
UMayor                        0.011050
universidad de valparaiso     0.005525
universidad de la serena      0.005525
universidad de concepcion     0.005525
titulada uandes               0.005525
me pertenezco a mi            0.005525
egresado universidad arcis    0.005525
duoc uc                       0.005525
chile                         0.005525
Name: institucionNormalizada, dtype: float64